In [3]:
##############
#K. David Roell CFPB 7/12/16
#Produces CSV outputs of counts for all macro edit fails
#
##############

import json
import os
import pandas as pd
import psycopg2
#from macro_sql import Q076 
#with open('macro_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
#    edit_sql = json.load(f)
macro_edits = ['Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q015', 'Q016', 'Q023', 'Q031', 'Q047', 'Q048',
              'Q053', 'Q054', 'Q055', 'Q056', 'Q057', 'Q058', 'Q061', 'Q062', 'Q063', 'Q065', 'Q070', 'Q071',
              'Q072', 'Q073', 'Q074', 'Q075', 'Q076', 'Q080', 'Q081', 'Q082', 'Q083']
two_table_edits = ['Q011', 'Q070', 'Q071', 'Q072', 'Q075', 'Q076']
#conn = psycopg2.connect("dbname=hmdamaster user=roellk") #connect and return connection
#cur = conn.cursor()#instantiate cursor object to use in SQL queries
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',

}

#connect_string = "dbname=%s user=%s host=%s password =%s" %(dbname, user, host, password) #set a string for connection to SQL
try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)



#edit_list = [item for item in dir(SQL_text) if not item.startswith("__")]

i'm connected


In [ ]:
year = 2014 #set start year
while year > 1992: #stop at last table that was loaded
    print(year, '\n', '*'*100)
    table = 'hmdalar' + str(year)
    table_prev = 'hmdalar' + str(year-1)
    first = True
    for edit in macro_edits:
        SQL="" #instantiate empty string

        with open('SQL_files/'+edit+'.sql') as f: #cycle over all SQL files in edit list
            for line in f.readlines():
                SQL = SQL + line.replace('\n', ' ') #add lines from SQL file to string

        SQL = SQL[1:] #remove Byte Order Marker (BOM) from string
        print(edit) #check SQL output
        print() #format spacing
        #if edit in group with 2 tables, execute with 2 table parameters
        if edit in two_table_edits:
            print(SQL.format(table=table, table_prev=table_prev))
            cur.execute(SQL.format(table=table, table_prev=table_prev)) #pass two tables to edits requiring it

        else:
            print(SQL.format(table=table))
            cur.execute(SQL.format(table=table)) #pass single table for most edits

        result_df = pd.DataFrame(cur.fetchall()) #convert query results to dataframe
        col_names = [desc[0] for desc in cur.description] #get list of column names from query result
        print(col_names)
        print(len(result_df.columns))
    #FIXME find out how to label empty df columns IE 58
    #FIXME add in context cols to SQL like denom and numerator
        if len(result_df.columns) == len(col_names): #check if DF is populated with correct number of columns
            print('setting col names')
            result_df.columns = col_names #change DF columns to result names

        if first == True and len(result_df.columns) > 0:
            print('seeding frame')
            first = False
            macro_df = result_df.copy() #seed dataframe with first edit results
        elif first == False and len(result_df.columns) > 0:
            print('merging frames')
            macro_df = macro_df.merge(result_df, how='outer', on=['arid', 'agency'])
        else:
            print("no data returned from query for {edit}".format(edit=edit))

        print(macro_df.head())
    macro_df.year = year
    path = 'macro_csvs'
    if not os.path.exists(path):
        os.makedirs(path)
    macro_df.to_csv('macro_'+str(year)+'.csv',index=False)
    year -=1

2014 
 ****************************************************************************************************
Q006

WITH numer AS (SELECT agency ,CONCAT(agency,rid) AS arid ,COUNT(sequence) as numer_count FROM hmdalar2014 WHERE action = '1' AND loan_purpose = '1' GROUP BY agency, CONCAT(agency, RID)),  denom AS (SELECT agency ,CONCAT(agency, RID) AS arid ,COUNT(sequence) as denom_count FROM hmdalar2014 WHERE loan_purpose = '1' GROUP BY agency, CONCAT(agency, rid))  SELECT numer.agency, numer.arid, (numer_count/denom_count::REAL) *100 AS Q006 FROM numer LEFT JOIN denom on numer.arid = denom.arid WHERE (numer_count/denom_count::REAL) * 100 > 95 AND numer_count > 25;


In [ ]:
#write loop to cycle over hmda macro csv years
#load to dictionary 'year':dataframe
#run script to get aggregates by agency code
#make graphs by agency code over time: 1 agency, edits over time

#FIXME this is broken due to changes above: exporting to new notebook
#load all annual CSVs to dictionary
#create new df copy for summary table
for key in macro_dfs.keys():
    summary_df = macro_dfs[key].copy() #copy df to create summary table
#    print(summary_df.head())
    #summary_df.drop(['q081_numer', 'q081_denom', 'q015_count'], axis=1, inplace=True)#drop unnecessary columns
    #summary_df = summary_df.rename(columns={'q015_value':'q015'}) #standardize names
    print(summary_df.head()) #view df for checking errors
    #get counts of macro edits by agency code
    by_agency_df = summary_df.groupby('agency').count() #produce counts of each edit type
    by_agency_df.drop('arid', axis=1, inplace=True) #drop arid

    #add total index and sum each edit count
    sum_df = by_agency_df.sum() #put sums of edits in new frame
    sum_df.name=('total') #name the frame (required to append)
    by_agency_df = by_agency_df.append(sum_df)
    by_agency_df['totals'] = by_agency_df.sum(axis=1)

    by_agency_df.to_csv('macro_by_agency_'+str(key)+'.csv') #write to file

In [22]:

by_agency_df #display df for error checking

,q006,q007,q008,q009,q010,q011,q015_count,q015_value,q016,q023,...,q074,q075,q076,q081_numer,q081_denom,q081,q082,q083,counts,totals
agency,,,,,,,,,,,,,,,,,,,,,
1,21,30,12,8,11,171,235,235,0,265,...,13,0,1,40,40,40,36,147,664,2027
2,18,16,8,3,4,82,172,172,0,167,...,3,0,1,25,25,25,24,142,420,1331
3,69,74,41,16,29,326,813,813,3,676,...,9,0,1,158,158,158,154,686,1906,6244
5,40,81,56,39,27,277,36,36,0,361,...,0,1,1,29,29,29,6,101,844,2144
7,18,25,59,41,31,323,18,18,0,66,...,70,6,9,4,4,4,1,14,501,1264
9,0,7,2,3,8,80,49,49,0,8,...,24,5,7,0,0,0,0,14,120,400
total,166,233,178,110,110,1259,1323,1323,3,1543,...,119,12,20,256,256,256,221,1104,4455,13410
